In [1]:
import os.path as osp
import torch.nn as nn
import torch
from focal_loss.focal_loss import FocalLoss

from torch.nn import TransformerEncoder, TransformerEncoderLayer
import os
from torch_geometric.data import Dataset, download_url
from torch_geometric.loader import DataLoader
import pandas as pd
from performance import performances_val
from PIL import Image
from torch.nn import Linear
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.data import Data
from sklearn import metrics
import numpy as np
from tqdm import tqdm
from torch_geometric.nn import ChebConv, GraphSAGE,GraphUNet ,TransformerConv

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
seed = 1378
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
landmark_colors = np.load("landmark_colors.npy")
from sklearn.preprocessing import OneHotEncoder
one_hot_list=[]
enc = OneHotEncoder()
enc.fit(np.expand_dims(landmark_colors,axis=1))  
for x in landmark_colors:
    one_hot_list.append(enc.transform([[x]]).toarray()[0])
one_hot_arr  = np.array(one_hot_list)
one_hot_arr = torch.FloatTensor(one_hot_arr)

m = torch.nn.Softmax(dim=-1)

class landmark_dataset(Dataset):
    def __init__(self, info_list, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(None, transform, pre_transform, pre_filter)
        
        self.graphs = pd.read_csv(info_list, delimiter=",", header=None)


    def len(self):
        return len(self.graphs)

    def get(self, idx):
        
        graph_path = self.graphs.iloc[idx]
        graph_path = graph_path[0]
        
        
        data = torch.load(graph_path)
        #data.x =data.x.t()
        return data 
class landmark_dataset_train(Dataset):
    def __init__(self, info_list, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(None, transform, pre_transform, pre_filter)
        
        self.graphs = pd.read_csv(info_list, delimiter=",", header=None)


    def len(self):
        return len(self.graphs)

    def get(self, idx):
        
        graph_path = self.graphs.iloc[idx]
        graph_path = graph_path[0]
        
        data = torch.load(graph_path)
        
 
        
        domain_name = graph_path.split("/")[-4]
        
            
        if domain_name == "MSU_MFSD" :
            
            domain_label = 0
            
        elif domain_name == "Replay_attack_dataset" :
            
            domain_label = 1
        else :
            print(graph_path,domain_name )
            raise "error"
            
        
        data = Data(x=data.x, edge_index=data.edge_index,y =data.y,data = domain_label,y_node = one_hot_arr)

        #data.x =data.x.t()
        return data 
train_dataset = landmark_dataset_train("/home/hassan-hossein/single_image_graph_face_anti_spoofing/train/graph_train_M_I_OS.txt")
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers=16,pin_memory=True)

test_dataset = landmark_dataset("/home/hassan-hossein/single_image_graph_face_anti_spoofing/train/graph_test_C_OS.txt")
test_loader = DataLoader(test_dataset, batch_size=64,num_workers=16)
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# فرض کنید که داده‌های ورودی شما یک توالی از ۶۸۰ ویژگی دارد.
#sequence_length = 128
#feature_dim = 4  # هر ویژگی تک‌بعدی است

# نمونه داده ورودی (تعداد نمونه‌ها × طول توالی × بعد ویژگی)
#inputs = torch.rand(32, sequence_length, feature_dim)  # 32 نمونه با 680 ویژگی تک‌بعدی

# تعریف یک لایه ترنسفورمر انکودر
class SimpleTransformer(nn.Module):
    def __init__(self, embed_dim, num_layers, nhead,n_out):
        super(SimpleTransformer, self).__init__()
        self.linear = nn.Linear(4, embed_dim)  # افزایش بعد ویژگی‌ها
        self.encoder_layer = TransformerEncoderLayer(d_model=embed_dim, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(128 * embed_dim, n_out)  # فرضا 10 کلاس خروجی داریم

    def forward(self, x):
        x = self.linear(x)  # تغییر بعد ویژگی‌ها
        x = self.transformer_encoder(x)
        x = x.view(x.size(0), -1)  # فشرده‌سازی به یک بردار برای طبقه‌بندی
        x = self.fc(x)
        return x

# # ایجاد مدل و ارسال ورودی
# # model = SimpleTransformer(embed_dim=embed_dim, num_layers=2, nhead=4)
# # output = model(inputs)

# #print(output.shape)
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# فرض کنید که داده‌های ورودی شما یک توالی از ۶۸۰ ویژگی دارد.
sequence_length = 128
feature_dim = 1  # هر ویژگی تک‌بعدی است

# نمونه داده ورودی (تعداد نمونه‌ها × طول توالی × بعد ویژگی)
#inputs = torch.rand(32, sequence_length, feature_dim)  # 32 نمونه با 680 ویژگی تک‌بعدی

# تعریف یک لایه ترنسفورمر انکودر
class SimpleTransformer_2(nn.Module):
    def __init__(self, embed_dim, num_layers, nhead):
        super(SimpleTransformer_2, self).__init__()
        self.linear = nn.Linear(feature_dim, embed_dim)  # افزایش بعد ویژگی‌ها
        self.encoder_layer = TransformerEncoderLayer(d_model=embed_dim, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(sequence_length * embed_dim, 7)  # فرضا 10 کلاس خروجی داریم

    def forward(self, x):
        x = self.linear(x)  # تغییر بعد ویژگی‌ها
        x = self.transformer_encoder(x)
        x = x.view(x.size(0), -1)  # فشرده‌سازی به یک بردار برای طبقه‌بندی
        x = self.fc(x)
        return x

# # ایجاد مدل و ارسال ورودی
# # model = SimpleTransformer(embed_dim=embed_dim, num_layers=2, nhead=4)
# # output = model(inputs)

# #print(output.shape)


In [3]:
class RandomNodeDropout(nn.Module):
    def __init__(self, drop_prob=0.3):
        super(RandomNodeDropout, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x, enabled=True):
        if not self.training or not enabled:
            return x
        
        num_rows = x.size(0)
        mask = torch.rand(num_rows, device=x.device) > self.drop_prob
        mask = mask.float().unsqueeze(1).expand_as(x)
        x = x * mask
        
        return x

soft = torch.nn.Softmax(dim=1)

num_domain = 2
class GRL(nn.Module):

    def __init__(self, max_iter):
        super(GRL, self).__init__()
        self.iter_num = 0
        self.alpha = 10
        self.low = 0.0
        self.high = 1.0
        self.max_iter = max_iter

    def forward(self, input):
        self.iter_num += 1
        return input * 1.0

    def backward(self, gradOutput):
        coeff = np.float(2.0 * (self.high - self.low) / (1.0 + np.exp(-self.alpha * self.iter_num / self.max_iter))
                         - (self.high - self.low) + self.low)
        return -coeff*10* gradOutput

num_node  =train_dataset.num_node_features
class Discriminator(torch.nn.Module):
    def __init__(self,hidden_channels ,max_iter):
        super(Discriminator, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv2 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv3 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv4 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.drop_1 = RandomNodeDropout(drop_prob=0.3)
        self.drop_2 = RandomNodeDropout(drop_prob=0.3)
        self.grl_layer = GRL(max_iter)
        self.fc2 = nn.Linear(hidden_channels, num_domain)
        
    def forward(self, x, edge_index, batch,drop_en):
        #print("graph fea :",graph_level_feature.shape ,edge_index.shape )
        
        x = self.grl_layer(x)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.drop_1(x,drop_en)
        
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.drop_2(x,drop_en)
        
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        
        x = global_mean_pool(x, batch)
        
        x = self.fc2(x)
        
        x = m(x)
         
        return x 



class GCN(torch.nn.Module):
    def __init__(self, hidden_channels,max_iter=4000):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = ChebConv(num_node, hidden_channels,2)
        self.conv2 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv3 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv4 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv5 = ChebConv(hidden_channels*2, hidden_channels,2)
        self.conv6 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv7 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv8 = ChebConv(hidden_channels, hidden_channels,2)
        self.lin = SimpleTransformer(8,5,8,2)
        
        self.drop_1 = RandomNodeDropout(drop_prob=0.3)
        self.drop_2 = RandomNodeDropout(drop_prob=0.3)
        self.drop_3 = RandomNodeDropout(drop_prob=0.3)
        self.drop_4 = RandomNodeDropout(drop_prob=0.3)
        self.drop_5 = RandomNodeDropout(drop_prob=0.3)
        self.drop_6 = RandomNodeDropout(drop_prob=0.3)
        
        
        self.b1 = BatchNorm(hidden_channels)
        self.b2 = BatchNorm(hidden_channels)
        self.b3 = BatchNorm(hidden_channels)
        self.b4 = BatchNorm(hidden_channels)
        self.b5 = BatchNorm(hidden_channels)
        self.b6 = BatchNorm(hidden_channels)
        self.b7 = BatchNorm(hidden_channels)
        
        self.dis = Discriminator(hidden_channels,max_iter)
        
        self.conv1_node_class = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        
        self.conv2_node_class = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        
        self.lin_node_class = SimpleTransformer_2(64,10,8)
        

        
        #self.tcn = Simple1DCNN()
        
    def forward(self, x, edge_index, batch,drop_en,index_arr):
        #print("graph fea :",graph_level_feature.shape ,edge_index.shape )
        

        x = self.conv1(x, edge_index)
        x  = self.b1(x)
        x = x.relu()
        
        x = self.drop_1(x,drop_en)
        
        

        x = self.conv2(x, edge_index)
        x  = self.b2(x)
        x = x.relu()
        
        x = self.drop_2(x,drop_en)
                
        dicriminator = x
        
        x_1 = global_mean_pool(x, batch)
        
        x = self.conv3(x, edge_index)
        x  = self.b3(x)
        x = x.relu()
        
        x = self.drop_3(x,drop_en)
        
        x_3 = global_mean_pool(x, batch)
        
        
        x = self.conv4(x, edge_index)
        x  = self.b4(x)
        x = x.relu()
        
        x = self.drop_4(x,drop_en)
        
        
        x = torch.cat((x,dicriminator),dim=1) #skip connection
        
        x = self.conv5(x, edge_index)
        x  = self.b5(x)
        x = x.relu()
        
        x = self.drop_5(x,drop_en)
        
        x_5 = global_mean_pool(x, batch)
        
        x = self.conv6(x, edge_index)
        x  = self.b6(x)
        x = x.relu()
        
        x = self.drop_6(x,drop_en)
        
        x = self.conv7(x, edge_index)
        x  = self.b7(x)
        x = x.relu()
        
        
        x = self.conv8(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        x = torch.stack([x,x_1,x_5,x_3],dim=2)
        
        x = F.dropout(x, p=0.5, training=self.training)
        
        #x = torch.unsqueeze(x,2)
        #print(x.shape)
        
        x = self.lin(x)
        
        x= m(x)
        
        
        node_class_hidden = self.conv1_node_class(dicriminator,edge_index)
        node_class_hidden = node_class_hidden.relu()
        
        node_class_hidden = self.conv2_node_class(node_class_hidden,edge_index)
        node_class_hidden = node_class_hidden.relu()
        
        #print(index_arr.shape , node_class_hidden.shape)
        
        node_class_hidden = node_class_hidden[index_arr]
        
        #print(node_class_hidden.shape)
        node_class_hidden = torch.unsqueeze(node_class_hidden, dim=2)
        
        node_pre = self.lin_node_class(node_class_hidden)
        
            
        #print("hiiii :" , dicriminator.shape)
        dis_invariant = self.dis(dicriminator,edge_index,batch,drop_en)
        
        return x ,dis_invariant ,node_pre

model = GCN(hidden_channels=128).to(device)
print(model)

def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#criterion = torch.nn.CrossEntropyLoss()
criterion= FocalLoss(gamma=2)
criterion_node_class=  torch.nn.CrossEntropyLoss()
criterion_2 = FocalLoss(gamma=2)
def train(epoch):
    model.train()
    losses_adv_loss=0
    losses_loss_cls=0
    total_loss = 0
    losses_node_cls_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}', unit='batch')
    for graph in progress_bar:  # Iterate in batches over the training dataset.
        graph = graph.to(device)  # Move data to the device
        
        drop_en = bool(np.random.binomial(1, 0.5))
        
        if graph.x.shape[0] < 29952 :
            
            if graph.x.shape[0] < 64:
            
                index_arr = torch.randint(0, int(graph.x.shape[0]), (int(graph.x.shape[0]),)).to(device)
            else :
                
                index_arr = torch.randint(0, int(graph.x.shape[0]), (64,)).to(device)
        else :
                
            index_arr = torch.randint(0, 29952, (64,)).to(device)
        
        
        
        out,domain_invariant,node_pre = model(graph.x, graph.edge_index, graph.batch,drop_en,index_arr)  # Perform a single forward pass.
        
        adv_loss = criterion_2(domain_invariant, graph.data)
        
        node_cls_loss = criterion_node_class(node_pre.squeeze(),graph.y_node.float()[index_arr])
        #print(out.shape , graph.y.long().shape)
        loss_cls = criterion(out.squeeze(), graph.y)  # Compute the loss.
        
        loss_all = adv_loss + loss_cls + node_cls_loss
        loss_all.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        total_loss = loss_all + total_loss
        losses_adv_loss = adv_loss + losses_adv_loss
        losses_loss_cls = loss_cls + losses_loss_cls
        losses_node_cls_loss = node_cls_loss + losses_node_cls_loss


    print("/////////////////////////")
    print("adverserial loss : " ,losses_adv_loss/len(train_loader.dataset))  
    print("classification loss : " , losses_loss_cls/len(train_loader.dataset))
    print("classification node loss : " , losses_node_cls_loss/len(train_loader.dataset))
    print("total loss : " ,total_loss/len(train_loader.dataset) )
    print("////////////////////////////////////")
def test(loader,epoch):
    score_path = os.path.join("/home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores", "epoch_{}".format(epoch+1))
    check_folder(score_path)
    model.eval()
    with torch.no_grad():
        scores_list = []

        correct = 0
        s= 0
        progress_bar = tqdm(loader, desc=f'Epoch {epoch+1}', unit='batch')
        for graph in progress_bar:  # Iterate in batches over the training/test dataset.
            if graph.x.shape[0] < 29952 :

                if graph.x.shape[0] < 64:

                    index_arr = torch.randint(0, int(graph.x.shape[0]), (int(graph.x.shape[0]),)).to(device)
                else :

                    index_arr = torch.randint(0, int(graph.x.shape[0]), (64,)).to(device)
            else :

                index_arr = torch.randint(0, 29952, (64,)).to(device)
                
            graph = graph.to(device)  # Move data to the device
            logit,_ ,_= model(graph.x, graph.edge_index, graph.batch,False,index_arr)
            #print(logit.shape)
            logit = logit[:,1]
            live_label = graph.y.float()
            for i in range(len(logit)):
                        scores_list.append("{} {}\n".format(logit[i].item(), live_label[i].item()))
  # Check against ground-truth labels.
    map_score_val_filename = os.path.join(score_path, "score.txt")
    with open(map_score_val_filename, 'w') as file:
                file.writelines(scores_list)
    print("score: write test scores to {}".format(map_score_val_filename))
    test_ACC, fpr, FRR, HTER, auc_test, test_err, tpr = performances_val(map_score_val_filename)
    print("epoch:{:d}, test:  val_ACC={:.4f}, HTER={:.4f}, AUC={:.4f}, val_err={:.4f}, ACC={:.4f}, TPR={:.4f}".format(
        epoch + 1, test_ACC, HTER, auc_test, test_err, test_ACC, tpr))  
    return auc_test




GCN(
  (conv1): ChebConv(640, 128, K=2, normalization=sym)
  (conv2): ChebConv(128, 128, K=2, normalization=sym)
  (conv3): ChebConv(128, 128, K=2, normalization=sym)
  (conv4): ChebConv(128, 128, K=2, normalization=sym)
  (conv5): ChebConv(256, 128, K=2, normalization=sym)
  (conv6): ChebConv(128, 128, K=2, normalization=sym)
  (conv7): ChebConv(128, 128, K=2, normalization=sym)
  (conv8): ChebConv(128, 128, K=2, normalization=sym)
  (lin): SimpleTransformer(
    (linear): Linear(in_features=4, out_features=8, bias=True)
    (encoder_layer): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
      )
      (linear1): Linear(in_features=8, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=8, bias=True)
      (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((8,), ep

In [3]:
best_auc = 0
for epoch in range(0, 1000):
    train(epoch)
    print("***************************")
    #print("train : ")
    #train_auc = test(train_loader,epoch)
    print("test : ")
    test_auc = test(test_loader,epoch)
    if test_auc > best_auc:
        print("improve acc .. .. ..")
        torch.save(model.state_dict(), 'two_domain_M_I_to_C_domain_generalization_tuning.pth')
        best_auc = test_auc
        continue  

Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2424/2424 [04:56<00:00,  8.18batch/s]


/////////////////////////
adverserial loss :  tensor(0.0003, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(0.0004, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(0.0183, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(0.0190, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 697/697 [00:56<00:00, 12.42batch/s]


score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_1/score.txt
epoch:1, test:  val_ACC=0.6395, HTER=0.3606, AUC=0.6679, val_err=0.3606, ACC=0.6395, TPR=0.0540
improve acc .. .. ..


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2424/2424 [05:15<00:00,  7.68batch/s]


...

/////////////////////////
adverserial loss :  tensor(2.8822e-09, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(3.0450e-07, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(0.0002, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(0.0002, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 176: 100%|████████████████████████████████████████████████████████████████████████████████████████| 697/697 [00:59<00:00, 11.78batch/s]


score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_176/score.txt
epoch:176, test:  val_ACC=0.6733, HTER=0.3268, AUC=0.7114, val_err=0.3268, ACC=0.6733, TPR=0.0859


Epoch 177:  63%|██████████████████████████████████████████████████████▎                               | 1531/2424 [03:24<01:59,  7.50batch/s]


KeyboardInterrupt: 

In [3]:
model.load_state_dict(torch.load("two_domain_M_I_to_C_domain_generalization_tuning.pth",map_location='cuda'))
best_auc = 0.8606
for epoch in range(82, 1000):
    train(epoch)
    print("***************************")
    #print("train : ")
    #train_auc = test(train_loader,epoch)
    print("test : ")
    test_auc = test(test_loader,epoch)
    if test_auc > best_auc:
        print("improve acc .. .. ..")
        torch.save(model.state_dict(), 'two_domain_M_I_to_C_domain_generalization_tuning.pth')
        best_auc = test_auc
        continue  

Epoch 83: 100%|███████████████████████████████████████████████████████████████████████████████████████| 1212/1212 [03:48<00:00,  5.31batch/s]


/////////////////////////
adverserial loss :  tensor(1.1684e-09, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(5.0146e-07, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(0.0002, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(0.0002, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 83: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 349/349 [01:03<00:00,  5.51batch/s]


score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_83/score.txt
epoch:83, test:  val_ACC=0.7612, HTER=0.2388, AUC=0.8384, val_err=0.2388, ACC=0.7612, TPR=0.3544


Epoch 84: 100%|███████████████████████████████████████████████████████████████████████████████████████| 1212/1212 [04:05<00:00,  4.95batch/s]


...

/////////////////////////
adverserial loss :  tensor(2.2009e-09, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(7.3312e-07, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 89: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 349/349 [00:57<00:00,  6.08batch/s]


score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_89/score.txt
epoch:89, test:  val_ACC=0.8160, HTER=0.1840, AUC=0.8867, val_err=0.1840, ACC=0.8160, TPR=0.5949


Epoch 90:   3%|██▋                                                                                      | 37/1212 [00:10<05:41,  3.44batch/s]


KeyboardInterrupt: 

In [4]:
model.load_state_dict(torch.load("two_domain_M_I_to_C_domain_generalization_tuning.pth",map_location='cuda'))
best_auc = 0.8961
for epoch in range(82, 1000):
    train(epoch)
    print("***************************")
    #print("train : ")
    #train_auc = test(train_loader,epoch)
    print("test : ")
    test_auc = test(test_loader,epoch)
    if test_auc > best_auc:
        print("improve acc .. .. ..")
        torch.save(model.state_dict(), 'two_domain_M_I_to_C_domain_generalization_tuning.pth')
        best_auc = test_auc
        continue  

Epoch 83: 100%|██████████████████████████| 1212/1212 [05:29<00:00,  3.68batch/s]


/////////////////////////
adverserial loss :  tensor(3.5422e-08, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(1.0763e-08, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 83: 100%|████████████████████████████| 349/349 [02:07<00:00,  2.73batch/s]


score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_83/score.txt
epoch:83, test:  val_ACC=0.8036, HTER=0.1965, AUC=0.8660, val_err=0.1964, ACC=0.8036, TPR=0.4650


Epoch 84: 100%|██████████████████████████| 1212/1212 [04:35<00:00,  4.40batch/s]


...

score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_108/score.txt
epoch:108, test:  val_ACC=0.7176, HTER=0.2824, AUC=0.7629, val_err=0.2824, ACC=0.7176, TPR=0.1369


Epoch 109: 100%|█████████████████████████| 1212/1212 [05:23<00:00,  3.75batch/s]


/////////////////////////
adverserial loss :  tensor(8.4598e-09, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(1.4627e-06, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(0.0001, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 109:  54%|██████████████▌            | 189/349 [01:11<01:00,  2.63batch/s]


KeyboardInterrupt: 

In [9]:
class RandomNodeDropout(nn.Module):
    def __init__(self, drop_prob=0.3):
        super(RandomNodeDropout, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x, enabled=True):
        if not self.training or not enabled:
            return x
        
        num_rows = x.size(0)
        mask = torch.rand(num_rows, device=x.device) > self.drop_prob
        mask = mask.float().unsqueeze(1).expand_as(x)
        x = x * mask
        
        return x

soft = torch.nn.Softmax(dim=1)

num_domain = 2
class GRL(nn.Module):

    def __init__(self, max_iter):
        super(GRL, self).__init__()
        self.iter_num = 0
        self.alpha = 10
        self.low = 0.0
        self.high = 1.0
        self.max_iter = max_iter

    def forward(self, input):
        self.iter_num += 1
        return input * 1.0

    def backward(self, gradOutput):
        coeff = np.float(2.0 * (self.high - self.low) / (1.0 + np.exp(-self.alpha * self.iter_num / self.max_iter))
                         - (self.high - self.low) + self.low)
        return -coeff*10* gradOutput

num_node  =train_dataset.num_node_features
class Discriminator(torch.nn.Module):
    def __init__(self,hidden_channels ,max_iter):
        super(Discriminator, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv2 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv3 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.conv4 = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        self.drop_1 = RandomNodeDropout(drop_prob=0.3)
        self.drop_2 = RandomNodeDropout(drop_prob=0.3)
        self.grl_layer = GRL(max_iter)
        self.fc2 = nn.Linear(hidden_channels, num_domain)
        
    def forward(self, x, edge_index, batch,drop_en):
        #print("graph fea :",graph_level_feature.shape ,edge_index.shape )
        
        x = self.grl_layer(x)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.drop_1(x,drop_en)
        
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.drop_2(x,drop_en)
        
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        
        x = global_mean_pool(x, batch)
        
        x = self.fc2(x)
        
        x = m(x)
         
        return x 



class GCN(torch.nn.Module):
    def __init__(self, hidden_channels,max_iter=4000):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = ChebConv(num_node, hidden_channels,2)
        self.conv2 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv3 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv4 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv5 = ChebConv(hidden_channels*2, hidden_channels,2)
        self.conv6 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv7 = ChebConv(hidden_channels, hidden_channels,2)
        self.conv8 = ChebConv(hidden_channels, hidden_channels,2)
        self.lin = SimpleTransformer(8,5,8,1)
        
        self.drop_1 = RandomNodeDropout(drop_prob=0.3)
        self.drop_2 = RandomNodeDropout(drop_prob=0.3)
        self.drop_3 = RandomNodeDropout(drop_prob=0.3)
        self.drop_4 = RandomNodeDropout(drop_prob=0.3)
        self.drop_5 = RandomNodeDropout(drop_prob=0.3)
        self.drop_6 = RandomNodeDropout(drop_prob=0.3)
        
        
        self.b1 = BatchNorm(hidden_channels)
        self.b2 = BatchNorm(hidden_channels)
        self.b3 = BatchNorm(hidden_channels)
        self.b4 = BatchNorm(hidden_channels)
        self.b5 = BatchNorm(hidden_channels)
        self.b6 = BatchNorm(hidden_channels)
        self.b7 = BatchNorm(hidden_channels)
        
        self.dis = Discriminator(hidden_channels,max_iter)
        
        self.conv1_node_class = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        
        self.conv2_node_class = GraphSAGE(hidden_channels, hidden_channels,2,dropout=0.5)
        
        self.lin_node_class = SimpleTransformer_2(64,10,8)
        

        
        #self.tcn = Simple1DCNN()
        
    def forward(self, x, edge_index, batch,drop_en,index_arr):
        #print("graph fea :",graph_level_feature.shape ,edge_index.shape )
        

        x = self.conv1(x, edge_index)
        x  = self.b1(x)
        x = x.relu()
        
        x = self.drop_1(x,drop_en)
        
        

        x = self.conv2(x, edge_index)
        x  = self.b2(x)
        x = x.relu()
        
        x = self.drop_2(x,drop_en)
                
        dicriminator = x
        
        x_1 = global_mean_pool(x, batch)
        
        x = self.conv3(x, edge_index)
        x  = self.b3(x)
        x = x.relu()
        
        x = self.drop_3(x,drop_en)
        
        x_3 = global_mean_pool(x, batch)
        
        
        x = self.conv4(x, edge_index)
        x  = self.b4(x)
        x = x.relu()
        
        x = self.drop_4(x,drop_en)
        
        
        x = torch.cat((x,dicriminator),dim=1) #skip connection
        
        x = self.conv5(x, edge_index)
        x  = self.b5(x)
        x = x.relu()
        
        x = self.drop_5(x,drop_en)
        
        x_5 = global_mean_pool(x, batch)
        
        x = self.conv6(x, edge_index)
        x  = self.b6(x)
        x = x.relu()
        
        x = self.drop_6(x,drop_en)
        
        x = self.conv7(x, edge_index)
        x  = self.b7(x)
        x = x.relu()
        
        
        x = self.conv8(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        x = torch.stack([x,x_1,x_5,x_3],dim=2)
        
        x = F.dropout(x, p=0.5, training=self.training)
        
        #x = torch.unsqueeze(x,2)
        #print(x.shape)
        
        x = self.lin(x)
        
        #x= m(x)
        
        x = x.sigmoid()
        
        
        node_class_hidden = self.conv1_node_class(dicriminator,edge_index)
        node_class_hidden = node_class_hidden.relu()
        
        node_class_hidden = self.conv2_node_class(node_class_hidden,edge_index)
        node_class_hidden = node_class_hidden.relu()
        
        #print(index_arr.shape , node_class_hidden.shape)
        
        node_class_hidden = node_class_hidden[index_arr]
        
        #print(node_class_hidden.shape)
        node_class_hidden = torch.unsqueeze(node_class_hidden, dim=2)
        
        node_pre = self.lin_node_class(node_class_hidden)
        
            
        #print("hiiii :" , dicriminator.shape)
        dis_invariant = self.dis(dicriminator,edge_index,batch,drop_en)
        
        return x ,dis_invariant ,node_pre

model = GCN(hidden_channels=128).to(device)
print(model)

def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCELoss()
#criterion= FocalLoss(gamma=2)
criterion_node_class=  torch.nn.CrossEntropyLoss()

criterion_2 = FocalLoss(gamma=2)

def train(epoch):
    model.train()
    losses_adv_loss=0
    losses_loss_cls=0
    total_loss = 0
    losses_node_cls_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}', unit='batch')
    for graph in progress_bar:  # Iterate in batches over the training dataset.
        graph = graph.to(device)  # Move data to the device
        
        drop_en = bool(np.random.binomial(1, 0.5))
        
        if graph.x.shape[0] < 29952 :
            
            if graph.x.shape[0] < 64:
            
                index_arr = torch.randint(0, int(graph.x.shape[0]), (int(graph.x.shape[0]),)).to(device)
            else :
                
                index_arr = torch.randint(0, int(graph.x.shape[0]), (64,)).to(device)
        else :
                
            index_arr = torch.randint(0, 29952, (64,)).to(device)
        
        
        
        out,domain_invariant,node_pre = model(graph.x, graph.edge_index, graph.batch,drop_en,index_arr)  # Perform a single forward pass.
        
        adv_loss = criterion_2(domain_invariant, graph.data)
        
        node_cls_loss = criterion_node_class(node_pre.squeeze(),graph.y_node.float()[index_arr])
        #print(out.shape , graph.y.long().shape)
        loss_cls = criterion(out.squeeze(), graph.y.float())  # Compute the loss.
        
        loss_all = adv_loss + loss_cls + node_cls_loss
        loss_all.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        total_loss = loss_all + total_loss
        losses_adv_loss = adv_loss + losses_adv_loss
        losses_loss_cls = loss_cls + losses_loss_cls
        losses_node_cls_loss = node_cls_loss + losses_node_cls_loss


    print("/////////////////////////")
    print("adverserial loss : " ,losses_adv_loss/len(train_loader.dataset))  
    print("classification loss : " , losses_loss_cls/len(train_loader.dataset))
    print("classification node loss : " , losses_node_cls_loss/len(train_loader.dataset))
    print("total loss : " ,total_loss/len(train_loader.dataset) )
    print("////////////////////////////////////")
def test(loader,epoch):
    score_path = os.path.join("/home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores", "epoch_{}".format(epoch+1))
    check_folder(score_path)
    model.eval()
    with torch.no_grad():
        scores_list = []

        correct = 0
        s= 0
        progress_bar = tqdm(loader, desc=f'Epoch {epoch+1}', unit='batch')
        for graph in progress_bar:  # Iterate in batches over the training/test dataset.
            if graph.x.shape[0] < 29952 :

                if graph.x.shape[0] < 64:

                    index_arr = torch.randint(0, int(graph.x.shape[0]), (int(graph.x.shape[0]),)).to(device)
                else :

                    index_arr = torch.randint(0, int(graph.x.shape[0]), (64,)).to(device)
            else :

                index_arr = torch.randint(0, 29952, (64,)).to(device)
                
            graph = graph.to(device)  # Move data to the device
            logit,_ ,_= model(graph.x, graph.edge_index, graph.batch,False,index_arr)
            #print(logit.shape)
            #logit = logit[:,1]
            live_label = graph.y.float()
            #print(logit.shape)
            for i in range(len(logit)):
                        scores_list.append("{} {}\n".format(logit[i].item(), live_label[i].item()))
  # Check against ground-truth labels.
    map_score_val_filename = os.path.join(score_path, "score.txt")
    with open(map_score_val_filename, 'w') as file:
                file.writelines(scores_list)
    print("score: write test scores to {}".format(map_score_val_filename))
    test_ACC, fpr, FRR, HTER, auc_test, test_err, tpr = performances_val(map_score_val_filename)
    print("epoch:{:d}, test:  val_ACC={:.4f}, HTER={:.4f}, AUC={:.4f}, val_err={:.4f}, ACC={:.4f}, TPR={:.4f}".format(
        epoch + 1, test_ACC, HTER, auc_test, test_err, test_ACC, tpr))  
    return auc_test




GCN(
  (conv1): ChebConv(640, 128, K=2, normalization=sym)
  (conv2): ChebConv(128, 128, K=2, normalization=sym)
  (conv3): ChebConv(128, 128, K=2, normalization=sym)
  (conv4): ChebConv(128, 128, K=2, normalization=sym)
  (conv5): ChebConv(256, 128, K=2, normalization=sym)
  (conv6): ChebConv(128, 128, K=2, normalization=sym)
  (conv7): ChebConv(128, 128, K=2, normalization=sym)
  (conv8): ChebConv(128, 128, K=2, normalization=sym)
  (lin): SimpleTransformer(
    (linear): Linear(in_features=4, out_features=8, bias=True)
    (encoder_layer): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
      )
      (linear1): Linear(in_features=8, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=8, bias=True)
      (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((8,), ep

In [6]:
for epoch in range(0, 1000):
    train(epoch)
    print("***************************")
    #print("train : ")
    #train_auc = test(train_loader,epoch)
    print("test : ")
    test_auc = test(test_loader,epoch)
    if test_auc > best_auc:
        print("improve acc .. .. ..")
        torch.save(model.state_dict(), 'three_domain_M_I_to_C_domain_generalization_tuning.pth')
        best_auc = test_auc
        continue  

Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2424/2424 [04:55<00:00,  8.20batch/s]


/////////////////////////
adverserial loss :  tensor(0.0003, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(0.0011, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(0.0182, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(0.0197, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 697/697 [01:01<00:00, 11.30batch/s]


score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_1/score.txt
epoch:1, test:  val_ACC=0.6323, HTER=0.3678, AUC=0.6737, val_err=0.3678, ACC=0.6323, TPR=0.0601
improve acc .. .. ..


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2424/2424 [05:11<00:00,  7.79batch/s]


...

/////////////////////////
adverserial loss :  tensor(9.8186e-12, device='cuda:1', grad_fn=<DivBackward0>)
classification loss :  tensor(3.2831e-09, device='cuda:1', grad_fn=<DivBackward0>)
classification node loss :  tensor(4.6309e-05, device='cuda:1', grad_fn=<DivBackward0>)
total loss :  tensor(4.6312e-05, device='cuda:1', grad_fn=<DivBackward0>)
////////////////////////////////////
***************************
test : 


Epoch 611: 100%|████████████████████████████████████████████████████████████████████████████████████████| 697/697 [00:58<00:00, 11.96batch/s]


score: write test scores to /home/hassan-hossein/single_image_graph_face_anti_spoofing/train/Scores/epoch_611/score.txt
epoch:611, test:  val_ACC=0.7380, HTER=0.2636, AUC=0.7626, val_err=0.2632, ACC=0.7380, TPR=0.0000


Epoch 612:  59%|██████████████████████████████████████████████████▍                                   | 1421/2424 [03:02<02:09,  7.77batch/s]


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load("two_domain_M_I_to_C_domain_generalization_tuning.pth",map_location='cuda'))

best_auc = 89.08
for epoch in range(0, 1000):
    train(epoch)
    print("***************************")
    #print("train : ")
    #train_auc = test(train_loader,epoch)
    print("test : ")
    test_auc = test(test_loader,epoch)
    if test_auc > best_auc:
        print("improve acc .. .. ..")
        torch.save(model.state_dict(), 'three_domain_M_I_to_C_domain_generalization_tuning.pth')
        best_auc = test_auc
        continue 

In [2]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# داده‌های ورودی (مثال فرضی)
# فرض کنید feature_list داده‌های ویژگی‌ها است
# و y_list لیبل‌ها، domain_list دامنه‌ها را نشان می‌دهد
# target_feature و target_label داده‌های هدف هستند

# اجرای TSNE برای کاهش ابعاد به 2
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(feature_list)

# رنگ‌های مختلف برای دامنه‌ها
colors = {0: 'blue', 1: 'green', 2: 'red'}

# رسم نمودار TSNE
plt.figure(figsize=(10, 10))

# رسم نقاط براساس دامنه‌ها و لیبل‌ها
for i, (x, y, domain, label) in enumerate(zip(X_tsne, y_list, domains, y_list)):
    if label == 0:
        marker = 'o'  # دایره برای کلاس 0
    else:
        marker = 'x'  # ضربدر برای کلاس 1
    plt.scatter(x[0], x[1], color=colors[domain], marker=marker, s=30, alpha=0.7)

# افزودن داده‌های هدف به نمودار
target_tsne = tsne.fit_transform(feature_list_target)
for x, label in zip(target_tsne, y_list_target):
    if label == 0:
        marker = 'o'  # دایره برای کلاس 0
    else:
        marker = 'x'  # ضربدر برای کلاس 1
    plt.scatter(x[0], x[1], color='yellow', marker=marker, s=30, edgecolor='black', linewidth=0.6, alpha=0.9)

# تنظیمات نمودار
plt.title("TSNE of Model Features by Class and Domain")
plt.xlabel("TSNE Component 1")
plt.ylabel("TSNE Component 2")
plt.legend(['MSU_MFSD', 'Replay_attack_dataset', 'Oulu', 'Target'], loc='upper right')
plt.grid(True)
plt.show()


NameError: name 'np' is not defined